<a href="https://colab.research.google.com/github/jibitesh2004/jibitesh2004/blob/main/Predict_rocket_launch_delays_with_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT LIBRARIES

In [24]:
import pandas as pd
import numpy as np
import requests  # For API calls

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import pydotplus
from IPython.display import display, clear_output, Image

import ipywidgets as widgets

# For voice output using gTTS
!pip install gTTS --quiet
from gtts import gTTS
from IPython.display import Audio

# For reproducibility
np.random.seed(42)


# **Load Historical Excel Data (Past Data)**


In [25]:
try:
    launch_data = pd.read_excel('Generated.xlsx')
    launch_data.columns = [col.strip() for col in launch_data.columns]  # Remove extra spaces
    print("Excel data loaded successfully.")
except Exception as e:
    print("Error loading Excel file. Using synthetic historical data for demo.", e)
    launch_data = pd.DataFrame({
        'Name': ['Rocket A'] * 10,
        'Date': pd.date_range(start='2025-01-01', periods=10),
        'Time (East Coast)': ['10:00 AM'] * 10,
        'Location': ['Cape Canaveral'] * 10,
        'Crewed or Uncrewed': np.random.choice(['Crewed', 'Uncrewed'], 10),
        'Launched?': np.random.choice(['Y', 'N'], 10),
        'Wind Direction': np.random.choice(['N', 'S', 'E', 'W'], 10),
        'Condition': np.random.choice(['Fair', 'Cloudy', 'Stormy'], 10)
    })

# Fill missing values
launch_data.fillna({'Launched?': 'N', 'Crewed or Uncrewed': 'Uncrewed',
                    'Wind Direction': 'unknown', 'Condition': 'Fair'}, inplace=True)

# Synthetic delay times and extra features
launch_data['DelayTime'] = np.random.uniform(10, 60, launch_data.shape[0])
launch_data['Temperature'] = np.random.uniform(15, 35, launch_data.shape[0])  # °C
launch_data['WindSpeed'] = np.random.uniform(0, 50, launch_data.shape[0])       # km/h
launch_data['Precipitation'] = np.random.uniform(0, 5, launch_data.shape[0])      # mm
launch_data['SensorReading'] = np.random.uniform(0.8, 1.0, launch_data.shape[0])  # Simulated sensor
launch_data['CommLog'] = np.random.choice([
    "All systems nominal, ready for launch.",
    "Minor issues detected in fuel sensors, proceed with caution.",
    "Crew on standby, monitoring weather.",
    "Unexpected alert in telemetry data, investigate further."
], launch_data.shape[0])


Excel data loaded successfully.


# **Clean & Process Historical Data**

In [26]:
# Define target and features
y = launch_data['DelayTime']
cols_to_drop = ['Name', 'Date', 'Time (East Coast)', 'Location', 'Launched?', 'DelayTime']
X = launch_data.drop(columns=cols_to_drop)

# Encode categorical features
label_encoder = LabelEncoder()
for col in ['Crewed or Uncrewed', 'Wind Direction', 'Condition']:
    X[col] = label_encoder.fit_transform(X[col])

numeric_features = ['Temperature', 'WindSpeed', 'Precipitation', 'SensorReading',
                      'Crewed or Uncrewed', 'Wind Direction', 'Condition']
text_feature = 'CommLog'

# Build pipelines for numeric and text data
numeric_pipeline = Pipeline([('scaler', StandardScaler())])
text_pipeline = Pipeline([('tfidf', TfidfVectorizer(max_features=50))])
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('text', text_pipeline, text_feature)
])

# Overall model pipeline
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=0, max_depth=5))
])

# Train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)
model_pipeline.fit(X_train, y_train)
r2_score = model_pipeline.score(X_test, y_test)
print(f"Model R² Score on Historical Test Data: {r2_score:.2f}")


Model R² Score on Historical Test Data: nan


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


# **Enrich Historical Data with Additional Features**

In [27]:
def tree_graph_to_png(tree_model, feature_names):
    tree_str = export_graphviz(tree_model,
                               feature_names=feature_names,
                               filled=True,
                               out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)
    return Image(graph.create_png())

feature_names = model_pipeline.named_steps['preprocessor'].get_feature_names_out()
decision_tree = model_pipeline.named_steps['regressor']
tree_image = tree_graph_to_png(decision_tree, feature_names=feature_names)
display(tree_image)


# **Define Features & Target for Training**

In [28]:
def fetch_live_weather(city="Cape Canaveral"):
    """
    Fetch live weather data using the Open-Meteo API (Temperature in Celsius).
    """
    latitude = 28.3922
    longitude = -80.6077
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        current_weather = data.get("current_weather", {})
        temperature = current_weather.get("temperature", None)
        wind_speed = current_weather.get("windspeed", None)
        precipitation = 0  # Not provided by current_weather
        return temperature, wind_speed, precipitation
    except Exception as e:
        print("Error fetching live weather:", e)
        return np.random.uniform(15, 35), np.random.uniform(0, 50), np.random.uniform(0, 5)

def simulate_live_space_condition():
    conditions = ["Calm", "Moderate", "Disturbed"]
    return np.random.choice(conditions)

def generate_sensor_alert(sensor_value, threshold=0.9):
    alert_message = ""
    if sensor_value < threshold:
        alert_message += "Alert: Fuel pump sensor reading is below normal. Check required. "
    else:
        alert_message += "Fuel pump sensor operating normally. "
    space_condition = simulate_live_space_condition()
    alert_message += f"Live space condition is {space_condition}."
    return alert_message

def fetch_space_weather(api_key="DEMO_KEY", event_type="FLR"):
    """
    Fetch space weather data from NASA's DONKI API (e.g., solar flares).
    Returns a summary string of the latest event.
    """
    base_url = f"https://api.nasa.gov/DONKI/{event_type}"
    params = {"api_key": api_key}
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        events = response.json()
        if events:
            event = events[0]
            event_summary = f"{event_type} on {event.get('beginTime', 'Unknown')}"
        else:
            event_summary = "No significant space weather events."
        return event_summary
    except Exception as e:
        print("Error fetching space weather:", e)
        return "Space weather data unavailable."

# Mappings for categorical features
crew_mapping = {'Crewed': 0, 'Uncrewed': 1}
wind_mapping = {'N': 0, 'S': 1, 'E': 2, 'W': 3, 'unknown': 4}
condition_mapping = {'Fair': 0, 'Cloudy': 1, 'Stormy': 2}


# **Encode Categorical Data for Training**

In [29]:
# --- Present Delay Functions ---
def generate_delay_statement(input_data, delay_minutes):
    reasons = []
    if input_data['WindSpeed'] > 30:
        reasons.append("high wind speeds")
    if input_data['Precipitation'] > 2:
        reasons.append("heavy precipitation")
    if input_data['SensorReading'] < 0.9:
        reasons.append("suboptimal sensor readings")
    keywords = ['alert', 'check required']
    if any(keyword in input_data['CommLog'].lower() for keyword in keywords):
        reasons.append("live sensor alert indicating potential issues")
    if not reasons:
        reasons.append("all parameters nominal")
    statement = (f"The rocket launch is predicted to be delayed by {delay_minutes:.1f} minutes "
                 f"({delay_minutes*60:.0f} seconds, {delay_minutes/60:.2f} hours) due to "
                 + ", ".join(reasons) + ".")
    return statement

def current_predict():
    # Build current input from live widgets (to be updated in interactive cell)
    auto_comm = generate_sensor_alert(sensor_slider.value)
    current_input_df = pd.DataFrame({
        'Temperature': [float(temp_label.value.split(":")[1].strip())],
        'WindSpeed': [wind_slider.value],
        'Precipitation': [precip_slider.value],
        'SensorReading': [sensor_slider.value],
        'Crewed or Uncrewed': [crew_mapping['Crewed']],
        'Wind Direction': [wind_mapping['N']],
        'Condition': [condition_mapping['Fair']],
        'CommLog': [auto_comm]
    })
    # Fetch latest space weather for display
    space_weather_status = fetch_space_weather()
    current_input_df["SpaceWeather"] = space_weather_status

    try:
        pred = model_pipeline.predict(current_input_df.drop(columns=['SpaceWeather']))[0]
    except Exception as e:
        print("Prediction error:", e)
        pred = 0.0
    explanation = generate_delay_statement(current_input_df.iloc[0].to_dict(), pred)
    return current_input_df, pred, explanation

# --- Future Delay Functions ---
def generate_future_sample():
    base_temp, base_wind, base_precip = fetch_live_weather()
    future_temp = np.clip(base_temp + np.random.normal(0, 1), 15, 35)
    future_wind = np.clip(base_wind + np.random.normal(0, 2), 0, 50)
    future_precip = np.clip(base_precip + np.random.normal(0, 0.5), 0, 10)
    future_sensor = np.random.uniform(0.8, 1.0)
    future_comm = generate_sensor_alert(future_sensor)
    future_space = fetch_space_weather()
    return {
        'Temperature': future_temp,
        'WindSpeed': future_wind,
        'Precipitation': future_precip,
        'SensorReading': future_sensor,
        'Crewed or Uncrewed': crew_mapping['Crewed'],
        'Wind Direction': wind_mapping['N'],
        'Condition': condition_mapping['Fair'],
        'CommLog': future_comm,
        'SpaceWeather': future_space
    }

def predict_future_delays(days=3, num_samples=100, delay_threshold=20):
    results = []
    for day in range(1, days+1):
        predictions = []
        for _ in range(num_samples):
            sample = generate_future_sample()
            sample_df = pd.DataFrame([sample])
            try:
                pred = model_pipeline.predict(sample_df.drop(columns=['SpaceWeather']))[0]
            except Exception as e:
                print("Future prediction error:", e)
                pred = 0.0
            predictions.append(pred)
        predictions = np.array(predictions)
        mean_delay = predictions.mean()
        std_delay = predictions.std()
        prob_exceed = np.mean(predictions > delay_threshold) * 100
        results.append({
            'Day': f"Day {day}",
            'Mean Delay (min)': round(mean_delay, 2),
            'Std Dev (min)': round(std_delay, 2),
            f"Prob(Delay > {delay_threshold} min) (%)": round(prob_exceed, 2)
        })
    return pd.DataFrame(results)


# **Build a Multimodal Pipeline (Regression)**

In [30]:
# --- Main Interactive Interface ---

# Mode selection: Present Delay or Future Delay
mode_selector = widgets.RadioButtons(
    options=['Present Delay', 'Future Delay'],
    description='Select Mode:',
    disabled=False
)

# Containers for each mode
present_box = widgets.VBox()
future_box = widgets.VBox()

# ----- PRESENT DELAY WIDGETS -----
temp_label = widgets.Label(value="Temperature (°C): --")
wind_slider = widgets.FloatSlider(min=0, max=50, step=0.5, description='Wind Speed (km/h):', value=0)
precip_slider = widgets.FloatSlider(min=0, max=10, step=0.1, description='Precipitation (mm):', value=0)
sensor_slider = widgets.FloatSlider(min=0.8, max=1.0, step=0.01, description='Sensor Reading:', value=0.85)

present_fetch_button = widgets.Button(description="Get Present Delay")
present_voice_output = widgets.Output()
present_message_output = widgets.Output()
present_data_output = widgets.Output()

present_get_message_button = widgets.Button(description="Get Message")
present_view_data_button = widgets.Button(description="View Data")

# Action for Present Delay: fetch live data and update prediction
def present_fetch_action(b):
    # Fetch live weather and update read-only label & sliders
    live_temp, live_wind, live_precip = fetch_live_weather()
    temp_label.value = f"Temperature (°C): {live_temp:.1f}"
    wind_slider.value = live_wind
    precip_slider.value = live_precip
    sensor_slider.value = np.random.uniform(0.8, 1.0)
    # Get current prediction
    current_input_df, pred, explanation = current_predict()
    # Display voice explanation
    with present_voice_output:
        clear_output(wait=True)
        try:
            tts = gTTS(text=explanation, lang='en')
            tts.save("present_explanation.mp3")
            display(Audio("present_explanation.mp3", autoplay=True))
        except Exception as e:
            print("Voice output error:", e)
    # Save results globally for message/data view
    present_box.results = {'df': current_input_df, 'pred': pred, 'explanation': explanation}

present_fetch_button.on_click(present_fetch_action)

def present_get_message_action(b):
    with present_message_output:
        clear_output(wait=True)
        if hasattr(present_box, 'results'):
            pred = present_box.results['pred']
            explanation = present_box.results['explanation']
            print("Prediction:", f"{pred:.1f} minutes delay "
                  f"({pred*60:.0f} seconds, {pred/60:.2f} hours)")
            print("Explanation:")
            print(explanation)
        else:
            print("No prediction available. Click 'Get Present Delay' first.")
present_get_message_button.on_click(present_get_message_action)

def present_view_data_action(b):
    with present_data_output:
        clear_output(wait=True)
        if hasattr(present_box, 'results'):
            temp_df = present_box.results['df'].copy()
            # Decode categorical values for readability
            temp_df['Crewed or Uncrewed'] = temp_df['Crewed or Uncrewed'].map({0: 'Crewed', 1: 'Uncrewed'})
            temp_df['Wind Direction'] = temp_df['Wind Direction'].map({0: 'N', 1: 'S', 2: 'E', 3: 'W', 4: 'unknown'})
            temp_df['Condition'] = temp_df['Condition'].map({0: 'Fair', 1: 'Cloudy', 2: 'Stormy'})
            print("----- Combined Data -----")
            display(temp_df)
        else:
            print("No data available. Click 'Get Present Delay' first.")
present_view_data_button.on_click(present_view_data_action)

present_box.children = [
    present_fetch_button,
    temp_label,
    wind_slider,
    precip_slider,
    sensor_slider,
    present_voice_output,
    present_get_message_button,
    present_message_output,
    present_view_data_button,
    present_data_output
]

# ----- FUTURE DELAY WIDGETS -----
future_day_input = widgets.IntText(value=3, description="Days Ahead:")
future_fetch_button = widgets.Button(description="Get Future Delay")
future_voice_output = widgets.Output()
future_message_output = widgets.Output()
future_output = widgets.Output()
future_get_message_button = widgets.Button(description="Get Future Text")

def future_fetch_action(b):
    # Get future prediction based on the number of days
    days = future_day_input.value
    future_df = predict_future_delays(days=days, num_samples=100, delay_threshold=20)
    with future_output:
        clear_output(wait=True)
        print(f"Future Delay Predictions for the Next {days} Day(s):")
        display(future_df)
    # Optionally, play a simple voice summary for future prediction
    with future_voice_output:
        clear_output(wait=True)
        summary_text = f"Future delay prediction for {days} days ahead completed."
        try:
            tts_future = gTTS(text=summary_text, lang='en')
            tts_future.save("future_explanation.mp3")
            display(Audio("future_explanation.mp3", autoplay=True))
        except Exception as e:
            print("Future voice error:", e)
future_fetch_button.on_click(future_fetch_action)

def future_get_message_action(b):
    with future_message_output:
        clear_output(wait=True)
        print("Future delay prediction details are displayed above in the table.")
future_get_message_button.on_click(future_get_message_action)

future_box.children = [
    future_day_input,
    future_fetch_button,
    future_voice_output,
    future_get_message_button,
    future_message_output,
    future_output
]

# --- Main Container: Display appropriate interface based on selection ---
main_container = widgets.VBox()

def update_mode(change):
    if change['new'] == 'Present Delay':
        main_container.children = [mode_selector, present_box]
    else:
        main_container.children = [mode_selector, future_box]

mode_selector.observe(update_mode, names='value')

# Set initial mode (default to Present Delay)
main_container.children = [mode_selector, present_box]

display(main_container)


ToggleButtons(description='Select Mode:', options=('Live Delay Prediction', 'Future Delay Prediction'), style=…

VBox()